### Working with Financial Data

Publicly traded companies are classified into one of 11 industry sectors, called GICS (Global Industry Classification Standard). Investors use these classes to design and construct portfolios, compare companies, evaluate industries, and isolate regional from global trends.

There are 11 GICS sectors

* Communication services
* Consumer discretionary
* Consumer staples
* Energy
* Financials
* Health care
* Industrials
* Information technology
* Materials
* Real estate
* Utilities


In this notebook we'll work with financial data, grouped by sector, to analyze different trends in companies and sectors listed on the S&P 500. 


Begin by loading the GICS sector mappings. They've been provided to you in a file called `gics.csv`


In [1]:
import pandas as pd

sectors = pd.read_csv('gics.csv')
sectors.head(10)

,Code,Name
0,50,Communication services
1,25,Consumer discretionary
2,30,Consumer staples
3,10,Energy
4,40,Financials
5,35,Health care
6,20,Industrials
7,45,Information technology
8,15,Materials
9,60,Real estate


Load the S&P 500 sector mappings and take a look at the data. The file is called `sp500-sector.csv`

In [2]:
sp_sectors = pd.read_csv('sp500-sector.csv')
sp_sectors.head()

,Symbol,Name,Sector
0,MMM,3M Company,Industrials
1,AOS,A.O. Smith Corp,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie Inc.,Health Care
4,ACN,Accenture plc,Information Technology


In [3]:
print(sp_sectors.Sector.unique())
print(sectors.Name.unique())

sp_sectors.Sector = sp_sectors.Sector.str.lower()
sectors.Name = sectors.Name.str.lower()

print(sp_sectors.Sector.unique())
print(sectors.Name.unique())

['Industrials' 'Health Care' 'Information Technology'
 'Consumer Discretionary' 'Utilities' 'Financials' 'Materials'
 'Real Estate' 'Consumer Staples' 'Energy' 'Telecommunication Services']
['Communication services' 'Consumer discretionary' 'Consumer staples'
 'Energy' 'Financials' 'Health care' 'Industrials'
 'Information technology' 'Materials' 'Real estate' 'Utilities']
['industrials' 'health care' 'information technology'
 'consumer discretionary' 'utilities' 'financials' 'materials'
 'real estate' 'consumer staples' 'energy' 'telecommunication services']
['communication services' 'consumer discretionary' 'consumer staples'
 'energy' 'financials' 'health care' 'industrials'
 'information technology' 'materials' 'real estate' 'utilities']


Create a dataframe that has the S&P 500 stock symbols with the appropriate sector number. For example, `3M`, symbol `MMM` is in `industrials` which corresponds to sector 20.

In [4]:
sp_sectors.set_index('Sector', inplace=True)
sectors.set_index('Name', inplace=True)
sectors.index.name = 'Sector'



In [5]:
print(sp_sectors.head())
print(sectors.head())

                       Symbol                 Name
Sector                                            
industrials               MMM           3M Company
industrials               AOS      A.O. Smith Corp
health care               ABT  Abbott Laboratories
health care              ABBV          AbbVie Inc.
information technology    ACN        Accenture plc
                        Code
Sector                      
communication services    50
consumer discretionary    25
consumer staples          30
energy                    10
financials                40


In [6]:
df = sp_sectors.join(sectors)
print(len(df))
print(len(sp_sectors))

df.head()


505
505


,Symbol,Name,Code
Sector,,,
consumer discretionary,AAP,Advance Auto Parts,25.0
consumer discretionary,AMZN,Amazon.com Inc.,25.0
consumer discretionary,APTV,Aptiv Plc,25.0
consumer discretionary,AZO,AutoZone Inc,25.0
consumer discretionary,BBY,Best Buy Co. Inc.,25.0


Working with strings in pandas can be much slower than working with integers and floats. Securities are typically represented by a security id, a numerical representation of the security. Ids have been provided in the file `sp500-ids.csv`. Load these into a dataframe and then generate a new dataframe that contains the columns `security_id` and `sector`.

In [7]:
ids = pd.read_csv('sp500-ids.csv')

final = ids.set_index('Symbol').join(df.set_index('Symbol'))
final.head()
final = final[['Security_id', 'Code']]
final.columns = ['security_id', 'sector']
final.index.name = 'symbol'
final.head()

,security_id,sector
symbol,,
MMM,1,20.0
AOS,2,20.0
ABT,3,35.0
ABBV,4,35.0
ACN,5,45.0


Load the file `sp500-5yr.csv` into a dataframe. This file contains 5 years worth of historical prices for securities listed on the S&P 500. Convert the symbols into security ids, and bring in the sector colums as well.

In [8]:
five_year = pd.read_csv('sp500-5yr.csv')
five_year.head()

,date,open,high,low,close,volume,symbol
0,2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
1,2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2,2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
3,2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
4,2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


Determine which sectors had the best performance for the past year, past three years and past five years. Note the start and end dates on the dataset.

In [9]:
df = five_year.set_index('symbol').join(final)
print(len(df))
print(len(five_year))
df.head()

619040
619040


,date,open,high,low,close,volume,security_id,sector
symbol,,,,,,,,
A,2013-02-08,45.07,45.35,45.00,45.08,1824755,15.0,35.0
A,2013-02-11,45.17,45.18,44.45,44.60,2915405,15.0,35.0
A,2013-02-12,44.81,44.95,44.50,44.62,2373731,15.0,35.0
A,2013-02-13,44.81,45.24,44.68,44.75,2052338,15.0,35.0
A,2013-02-14,44.72,44.78,44.36,44.58,3826245,15.0,35.0


For each sector, determine the top performer and the bottom performer for the 1 year, 3 year, and 5 year windows.

In [10]:
df.date = pd.to_datetime(df.date)

df = df.reset_index().set_index(['date', 'security_id'])
df = df.sort_index()


df.head(10)

symbol      open      high      low     close  \
date       security_id                                                 
2013-02-08 1.0            MMM  102.3400  103.3300  102.040  102.6600   
           2.0            AOS   17.3825   17.5675   17.340   17.4875   
           3.0            ABT   34.3900   34.6600   34.290   34.4100   
           4.0           ABBV   36.3700   36.4200   35.825   36.2500   
           5.0            ACN   73.0100   73.7100   72.820   73.3100   
           6.0           ATVI   13.2000   13.9430   12.900   13.4100   
           7.0            AYI   69.2100   70.7000   69.210   70.0900   
           8.0           ADBE   38.3100   39.4500   38.145   39.1200   
           9.0            AAP   78.3400   79.7200   78.010   78.9000   
           10.0           AMD    2.5800    2.6000    2.580    2.5900   

                          volume  sector  
date       security_id                    
2013-02-08 1.0           1937721    20.0  
           2.0           1138412    20.0  
           3.0          10237828    35.0  
           4.0          13858795    35.0  
           5.0           2000477    45.0  
           6.0          56017540    45.0  
           7.0            265713    20.0  
           8.0           5104545    45.0  
           9.0           1298137    25.0  
           10.0          5971768    45.0

In [11]:
start = df['2013-02-08': '2013-02-08']
start = start.groupby('sector')['open'].sum()

end = df['2014-02-07': '2014-02-07']
end = end.groupby('sector')['close'].sum()

print('One Year')
print(end - start)

print("\n\nThree Year")
end = df['2015-02-06': '2015-02-06']
end = end.groupby('sector')['close'].sum()
print(end - start)


print("\n\nFive Year")
end = df['2018-02-07' : '2018-02-07']
end = end.groupby('sector')['close'].sum()
print(end - start)

One Year
sector
10.0     285.4250
15.0     210.5320
20.0    1138.5556
25.0    1514.7884
30.0     233.1200
35.0    1395.5484
40.0     772.7788
45.0     878.5006
55.0      71.8950
60.0      19.4500
dtype: float64


Three Year
sector
10.0     170.4600
15.0     451.5030
20.0    1836.5766
25.0    2301.3158
30.0     656.0667
35.0    2732.1934
40.0    1369.5334
45.0    1936.5362
55.0     341.3400
60.0     638.2300
dtype: float64


Five Year
sector
10.0      26.1600
15.0    1177.9900
20.0    4261.6078
25.0    4229.7874
30.0    1141.7167
35.0    4648.0419
40.0    3327.3544
45.0    5683.9413
55.0     384.1200
60.0     718.7900
dtype: float64


Imagine two portfolios, one comprised of the securites represented by even ids and one by odd ids. Which portfolio has had the best performance over the 1,3 and 5 year time windows?


In [12]:
data = df.reset_index()

even = data.loc[data.security_id % 2 == 0] 
odd = data.loc[data.security_id %2 == 1]
even = even.dropna()
odd = odd.dropna()

even.set_index('date', inplace=True)
odd.set_index('date', inplace=True)

starte = even['2013-02-08': '2013-02-08']
starte = starte.groupby('sector')['open'].sum()

starto = odd['2013-02-08': '2013-02-08']
starto = starto.groupby('sector')['open'].sum()


ende = even['2014-02-07': '2014-02-07']
ende = ende.groupby('sector')['close'].sum()

endo = odd['2014-02-07': '2014-02-07']
endo = endo.groupby('sector')['close'].sum()

print('One Year, Even')
print(ende - starte)

print('One Year, Odd')
print(endo - starto)

print("\n\nThree Year, Even")
ende = even['2015-02-06': '2015-02-06']
ende = ende.groupby('sector')['close'].sum()

endo = odd['2015-02-06': '2015-02-06']
endo = endo.groupby('sector')['close'].sum()

print(ende - starte)
print("\n\nThree Year, Odd")
print(endo - starto)


print("\n\nFive Year, Even")
ende = even['2018-02-07' : '2018-02-07']
ende = ende.groupby('sector')['close'].sum()
endo = odd['2018-02-07' : '2018-02-07']
endo = endo.groupby('sector')['close'].sum()
print(ende - starte)

print("\n\nFive Year, Odd")
print(endo - starto)




One Year, Even
sector
10.0    139.0400
15.0     97.5950
20.0    344.6122
25.0    674.8203
30.0    129.1550
35.0    728.4300
40.0    352.1468
45.0    523.7284
55.0     26.4650
60.0     -6.3700
dtype: float64
One Year, Odd
sector
10.0    146.3850
15.0    112.9370
20.0    793.9434
25.0    839.9681
30.0    103.9650
35.0    667.1184
40.0    420.6320
45.0    354.7722
55.0     45.4300
60.0     25.8200
dtype: float64


Three Year, Even
sector
10.0     118.8100
15.0     192.6100
20.0     490.4682
25.0    1075.7327
30.0     372.0300
35.0    1395.9650
40.0     663.2484
45.0     710.6798
55.0     205.8000
60.0     281.2500
dtype: float64


Three Year, Odd
sector
10.0      51.6500
15.0     258.8930
20.0    1346.1084
25.0    1225.5831
30.0     284.0367
35.0    1336.2284
40.0     706.2850
45.0    1225.8564
55.0     135.5400
60.0     356.9800
dtype: float64


Five Year, Even
sector
10.0     123.1400
15.0     601.6250
20.0    1420.2844
25.0    1857.8193
30.0     580.8800
35.0    2599.4250
40.0    1680.

Combine these two portfolios into a single portfolio. What was the overall performance of the SP500 for the same time periods?


In [13]:
pd.concat([endo - starto, ende - starte]).sum()


25599.509500000007